Step 1 - Data Engineering

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import csv

In [2]:
# Read the CSV data into DataFrames
stations="hawaii_stations.csv"
weather="hawaii_measurements.csv"
stations_data=pd.read_csv(stations,encoding="iso-8859-1")
weather_data=pd.read_csv(weather,encoding="iso-8859-1")
stations_df=pd.DataFrame(stations_data)
weather_df=pd.DataFrame(weather_data)

In [3]:
# Check missing data points.. NaN's probably in 'prcp' column.  
weather_df.count()

station    19550
date       19550
prcp       18103
tobs       19550
dtype: int64

In [33]:
# View the data 
weather_df.tail()

,station,date,prcp,tobs
19545,USC00516128,2017-08-19,0.09,71
19546,USC00516128,2017-08-20,NaN,78
19547,USC00516128,2017-08-21,0.56,76
19548,USC00516128,2017-08-22,0.50,76
19549,USC00516128,2017-08-23,0.45,76


In [5]:
# View the numbers
weather_df.describe()

,prcp,tobs
count,18103.000000,19550.000000
mean,0.160644,73.097954
std,0.468746,4.523527
min,0.000000,53.000000
25%,0.000000,70.000000
50%,0.010000,73.000000
75%,0.110000,76.000000
max,11.530000,87.000000


In [6]:
#Cleaning data by replacing NaNs with 0 instead of deleting. The prcp values are very low 
clean_weather_df = weather_df.fillna(0)
clean_weather_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-06,0.00,73


In [7]:
# Review the numbers .. check the differences after data clean up
clean_weather_df.describe()

,prcp,tobs
count,19550.000000,19550.000000
mean,0.148753,73.097954
std,0.453021,4.523527
min,0.000000,53.000000
25%,0.000000,70.000000
50%,0.010000,73.000000
75%,0.090000,76.000000
max,11.530000,87.000000


In [8]:
# Data cleaned and ok to proceed with saving to CSV
clean_weather_df.to_csv("clean_weather.csv, index=False")

In [9]:
# View missing data points.
stations_df.count()

station      9
name         9
latitude     9
longitude    9
elevation    9
dtype: int64

In [10]:
# View the data 
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [11]:
# View the data 
stations_df.describe()

,latitude,longitude,elevation
count,9.000000,9.000000,9.000000
mean,21.393826,-157.867098,60.977778
std,0.086442,0.103873,103.465547
min,21.271600,-158.011100,0.900000
25%,21.333100,-157.975100,7.000000
50%,21.393400,-157.837400,14.600000
75%,21.451670,-157.802500,32.900000
max,21.521300,-157.711390,306.600000


In [12]:
# Data clean and ok to proceed with saving to CSV
clean_stations_df = stations_df

In [13]:
clean_stations_df.to_csv("clean_stations.csv, index=False")

Step 2 - Database Engineering

In [14]:
# Creating tables in SQL .. using python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [15]:
# Creating an engine to a database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [16]:
# Creating connection to engine
conn = engine.connect()

In [17]:
# Use `declarative_base` to model tables as an ORM class
Base = declarative_base()

In [18]:
class measurements(Base):
    __tablename__ = 'measurements'
#     __table_args__ = {'extend_existing': True} # replaces existing metadata for measurements table/class
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, station={self.station}, prcp={self.prcp}"

In [19]:
class stations(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key = True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f" id = {self.id}, station = {self.station}, name = {self.name}"

In [20]:
# Create the 2 tables in hawaii.sqlite database
Base.metadata.create_all(engine)

In [21]:
# View stations table schema
stations.__table__

Table('stations', MetaData(bind=None), Column('id', Integer(), table=<stations>, primary_key=True, nullable=False), Column('station', Text(), table=<stations>), Column('name', Text(), table=<stations>), Column('latitude', Float(), table=<stations>), Column('longitude', Float(), table=<stations>), Column('elevation', Float(), table=<stations>), schema=None)

In [22]:
# View measurements table schema
measurements.__table__

Table('measurements', MetaData(bind=None), Column('id', Integer(), table=<measurements>, primary_key=True, nullable=False), Column('station', Text(), table=<measurements>), Column('date', Text(), table=<measurements>), Column('prcp', Float(), table=<measurements>), Column('tobs', Float(), table=<measurements>), schema=None)

In [23]:
# Convert clean dataframe(without NaNs) to dictionary using orient value as records (column -> value)
weather_dict = clean_weather_df.to_dict(orient='records')
print(weather_dict[0]) # View if conversion worked 

{'station': 'USC00519397', 'date': '2010-01-01', 'prcp': 0.08, 'tobs': 65}


In [24]:
# converting stations_df to dictionary# convert 
stations_dict = stations_df.to_dict(orient='records')
print(stations_dict[0]) #checking the conversion worked ok

{'station': 'USC00519397', 'name': 'WAIKIKI 717.2, HI US', 'latitude': 21.2716, 'longitude': -157.8168, 'elevation': 3.0}


Load dictionaries into SQL tables

In [25]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [26]:
# Create a variable for measurements table, to load it into hawaii.sqlite later
measure_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [27]:
# Create a variable for each stations table, to load it into sqlite later
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [28]:
# Need to delete previously existing rows/data by using `table.delete()` to remove any pre-existing data and reload table
conn.execute(measure_table.delete())

In [29]:
# Insert csv data that was converted into dictionaries in tables variables created above
conn.execute(measure_table.insert(), weather_dict)

In [30]:
# Test inserted data by querying first 5 rows
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-06', 0.0, 73.0)]

In [31]:
# Insert csv data that was converted into dictionaries in tables variables created above
conn.execute(stations_table.insert(), stations_dict)

In [32]:
# Test inserted data by querying first 5 rows
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]